# Disclaimer
This workshop aims at making the OT-2 APIv2 as approachable as possible for people like me who have limited experience in programming, and therefore: 
1. Coding practices may be unorthodox and even heretic in the eyes of properly trained programmers.
2. Methods of execution are in most instances suboptimal in performance. (see Additional Remarks #1)


API = APIv2

# Intended Outcomes

By the end of the workshop, we hope participants would be able to:

1.	Read the documentation on APIv2, and know where to look for more information, and where to seek help if needed
2.	Set up an environment for drafting a Python protocol for OT-2 APIv2
3.	Import the opentrons library to initialize a protocol object, create labware objects and pipettes within it and retrieve their information
4.	Access wells of plates / racks in different orders and using different methods
5.	Instruct the pipettes to carry out liquid movement using pipette.transfer() and pipette.distribute(), and configure them with additional parameters
6.	Create a custom pipetting step using atomic commands, exemplified by a wetting step.
7.	Simulate the protocol in a PC and then converting it to a robot-executable one
8.	Debug the code using the variable explorer and the console
9.	Avoid codes that could rig the protocol or could complicate debugging
10.	Use loops, lists and dictionaries together to scale up pipetting steps
11.	Use Excel to create list of parsable commands to be copied and pasted into a protocol
12.	Locate information for converting scripts written in APIv1 into APIv2-coded scripts  

13.	Write the framework of practically useful protocols for automatable experiments

# System Setup

## Install the IDE
It is best to make use of an Integrated Development Environment (IDE) when writing codes. My personal experience suggests that  Spyder is most friendly to Python beginners and its Variable Explorer is tremendously helpful, both in debugging and in understanding how the code works. Some other workshops recommend Jupyter or PyCharm. Participants are welcomed to stick with the IDEs they are most comfortable with.

The relatively fool-proof way to get Spyder is to download and install [Anaconda](https://www.anaconda.com/distribution/). For advanced users, the accompanying site packages like pandas and NumPy comes in handy for interfacing experiment results and experiment automation. (This will not be covered here)

## Install the opentrons module

Once Anaconda is installed, participants should open Spyder, and install the opentrons module / library, by typing in the IPython console:  

`pip install opentrons`

To check if the opentrons is correctly installed, participants should try and see if they could get the module version from the  console.

In [18]:
import opentrons
print(opentrons.__version__)

3.16.1


Failure to do so should give an error message of `ModuleNotFoundError: No module named 'opentrons'`

## Keeping the opentrons module up-to-date
Opentrons update their API and library from time to time and it should happen together with OT-2 App and robot firmware upgrade. Always check if your opentrons module is the latest, and if needed, upgrade your opentrons module on your computer by typing in the terminal  

`pip install opentrons --upgrade`

# Setting up a simulation environment for drafts

## Background
Starting from OT-2 APIv2, Opentrons had separated modules for simulation and 

In the OT-2 APIv2, an executable protocol on the robot demands that all steps be wrapped inside a `def run()` function. This severely complicates debugging and renders the Spyder Variable Explorer useless. Thus, jumping directly to protocol building is discouraged in this workshop.


The Opentrons APIv2, unlike it predecessor APIv1, does not allow simulation in the protocol in loco.


# Additional remarks

1. It is faster to use `pipette.transfer()` with list of well objects than looping a a single `pipette.transfer()` function multiple times. The difference in speed in unnoticeable on a PC but could mean 4-5 minutes of wait when API of a long protocol is compiled on a Pi-powered robot.

The following example illustrates a comparison of the two methods, both trying to pipette 200 µL from well A1 to A7, B1 to B7, ... H6 to H12, for a total of 48 transfer steps.

A single complex function processes multi-to-multi transfers much more efficiently, at the expense of losing fine-tuned control at the atomic movement level.

In [1]:
# Set up
import time
from opentrons import simulate
protocol = simulate.get_protocol_api('2.2')
tip_rack = protocol.load_labware('opentrons_96_tiprack_300ul', location = '1')
plate = protocol.load_labware('corning_96_wellplate_360ul_flat', location = '2')
pipette = protocol.load_instrument('p300_single', mount = 'right', tip_racks = [tip_rack])
print('Run time comparison:')

# Method 1: List for looping pipette.transfer
transfer_info = {x: x + 48 for x in range(0,48)}

start_time = time.time()
for source_index, dest_index in transfer_info.items():
    pipette.transfer(200, plate.wells(source_index), plate.wells(dest_index))
end_time = time.time()
time_diff = end_time - start_time
print('Loop method: ' + str(time_diff))

# Method 2: Looping through a series of wells
source_wells = plate.columns()[0:3]
dest_wells = plate.columns()[4:7]
start_time = time.time()
pipette.transfer(200, source_wells, dest_wells)
end_time = time.time()
time_diff = end_time - start_time
print('Well object method: ' + str(time_diff))

Loading json containers...
Json container file load complete, listing database
Found 0 containers to add. Starting migration...
Database migration complete!


C:\Users\s1635543\.opentrons\deck_calibration.json not found. Loading defaults
C:\Users\s1635543\.opentrons\robot_settings.json not found. Loading defaults
C:\Users\s1635543\.opentrons\deck_calibration.json not found. Loading defaults
C:\Users\s1635543\.opentrons\robot_settings.json not found. Loading defaults


Run time comparison:
Loop method: 0.6629588603973389
Well object method: 0.11969971656799316


2. To interface with external files like .csv, the API provides a method but that is not recommended.
That is however not recommended.